# Web Scraping Google Search Results

In [1]:
import pandas as pd
import numpy as np
import time
import random
import regex as re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

# Custom functions
from my_functions import *

## Web Scrape Mojo Website
Form initial list of movies based off the Box Office Mojo website

In [2]:
# Import Box Office Mojo Movie list
df_movies = pd.read_csv('bk_mojo.csv', dtype=object)
df_movies.head()

,Title,Worldwide Lifetime Gross,Domestic Lifetime Gross,Domestic %,Foreign Lifetime Gross,Foreign %,Year
0,Avatar,"$2,847,246,203","$760,507,625",26.7%,"$2,086,738,578",73.3%,2009
1,Avengers: Endgame,"$2,797,501,328","$858,373,000",30.7%,"$1,939,128,328",69.3%,2019
2,Titanic,"$2,201,647,264","$659,363,944",30%,"$1,542,283,320",70%,1997
3,Star Wars: Episode VII - The Force Awakens,"$2,069,521,700","$936,662,225",45.3%,"$1,132,859,475",54.7%,2015
4,Avengers: Infinity War,"$2,048,359,754","$678,815,482",33.1%,"$1,369,544,272",66.9%,2018


## Web Scrape Google Search Results

### First search of movies

In [3]:
movies = df_movies.loc[:, ['Title', 'Year']]
print('Number of movies: ', len(df_movies))

Number of movies:  1000


In [4]:
%%time

lst_scores = []
lst_like = []
lst_g_rev = []
lst_dist = []
lst_combined= []
loop_rand = random.randrange(10, 40)
loop_reset = 1
loop_counter = 1

driver = webdriver.Chrome('./chromedriver')
driver.get("https://www.google.com/")

# df_movies = missing_movies

for index, movie in df_movies.iterrows():

    # Print stage in loop every 25 movies
    if loop_counter % 25 == 0:
        print('Loops completed: ', loop_counter)
    loop_counter += 1

    search = driver.find_element_by_name('q')
    search.send_keys(movie['Title']+ ' film '+  movie['Year'])
    search.send_keys(Keys.RETURN)

    # Movie review scores
    try:
        lst = google_scrape('//*[@id="kp-wp-tab-overview"]/div[1]/div/div/div[2]/div/div[1]/div[1]', driver)
        lst1_dict = [{'Title': movie['Title']
                    , 'Year': movie['Year']
                    , lst[1] : lst[0]
                    , lst[3] : lst[2]
                    , lst[5] : lst[4]}]
        lst_scores.append(lst1_dict)
    except:
        pass

    # Did Google users like the movie?
    try:
        lst1_like = google_scrape('//*[@id="kp-wp-tab-overview"]/div[1]/div/div/div[2]/div/div[2]/div/div[1]/div', driver)
        lst_like.append(lst1_like)
    except:
        pass

    # Google User Reviews Score
    for i in [2, 3, 4,5,6]:
        try: 
            path = '//*[@id="kp-wp-tab-overview"]/div[' + str(i) + ']/div/div/div/div/div/div/div/div[2]/div[2]'
            lst1_g_rev = google_scrape(path, driver)
            lst_g_rev.append(lst1_g_rev)
            break
        except:
            pass    

    # Film description (from Google)
    try:
        lst1_dist = google_scrape('//*[@id="kp-wp-tab-overview"]/div[1]/div/div/div[2]/div/div[3]/div/div/div', driver)
        lst_dist.append(lst1_dist)
    except:
        pass    

    # Combine everything except the agggregated movie reviews
    lst_combined.append([movie['Title']
                        , movie['Year']
                        , lst1_like
                        , lst1_g_rev
                        , lst1_dist])
                        #, lst1_dict])

    # Empty lists
    lst1_like = []
    lst1_g_rev = []
    lst1_dist = []
    lst1_dict = []

    # Clear search bar
    driver.find_element_by_name('q').clear()

    # Wait random amount of time
    time.sleep(np.random.random() * 5)

    # Close and re-open driver at random interval to try to avoid Google's block
    loop_reset += 1
    if loop_reset >= loop_rand:
        loop_rand = random.randrange(20, 50)
        loop_reset = 1

        driver.close()
        driver = webdriver.Chrome('./chromedriver')
        driver.get("https://www.google.com/")

driver.close()


<timed exec>:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
<timed exec>:22: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
<timed exec>:77: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead


Loops completed:  25


<timed exec>:89: DeprecationWarning: executable_path has been deprecated, please pass in a Service object


Loops completed:  50
Loops completed:  75
Loops completed:  100
Loops completed:  125
Loops completed:  150
Loops completed:  175
Loops completed:  200
Loops completed:  225
Loops completed:  250
Loops completed:  275
Loops completed:  300
Loops completed:  325
Loops completed:  350
Loops completed:  375
Loops completed:  400
Loops completed:  425
Loops completed:  450
Loops completed:  475
Loops completed:  500
Loops completed:  525
Loops completed:  550
Loops completed:  575
Loops completed:  600
Loops completed:  625
Loops completed:  650
Loops completed:  675
Loops completed:  700
Loops completed:  725
Loops completed:  750
Loops completed:  775
Loops completed:  800
Loops completed:  825
Loops completed:  850
Loops completed:  875
Loops completed:  900
Loops completed:  925
Loops completed:  950
Loops completed:  975
Loops completed:  1000
Wall time: 1h 17min 33s


In [5]:
col_names = ['Title'
             , 'Year'
             , 'Google_Users'
             , 'Google user ratings'
             , 'movie description']

df_google = pd.DataFrame(lst_combined, columns = col_names)

# Backup Movie list
df_google.to_csv('bk_google_1_raw.csv', index = False)

df_google.tail()

,Title,Year,Google_Users,Google user ratings,movie description
995,Operation Mekong,2016,"[85% liked this film, Google users]",[],"[Description, The Chinese government launches ..."
996,Anchorman 2: The Legend Continues,2013,"[83% liked this film, Google users]","[4.0, 93 ratings]","[Description, A celebrated news anchor from Sa..."
997,Creed,2015,"[91% liked this film, Google users]","[4.6, 619 ratings]","[Description, Adonis Johnson, the son of heavy..."
998,Gemini Man,2019,"[75% liked this film, Google users]","[3.8, 2583 ratings]","[Description, Henry is an ageing assassin who ..."
999,The Girl on the Train,2016,"[77% liked this film, Google users]","[3.6, 872 ratings]","[Description, Rachel, a divorcee, idealises Sc..."


### Missing Movies to search for again

In [6]:
missing_movies = []

for index, movie in df_google.iterrows():
    if (movie['Google_Users'] == [] 
        or movie['Google user ratings'] == []
        or movie['movie description'] == []):
        missing_movies.append(movie) 
        
missing_movies = pd.DataFrame(missing_movies)# .loc[:, ['Title', 'Year']]
missing_movies['Year'] = missing_movies['Year'].astype('str') 
print(len(missing_movies), 'movies to run search for again')
missing_movies.head()

61 movies to run search for again


,Title,Year,Google_Users,Google user ratings,movie description
65,Ice Age: Dawn of the Dinosaurs,2009,"[89% liked this film, Google users]",[],"[Description, Manny and Ellie are expecting th..."
83,Wonder Woman,2017,"[84% liked this film, Google users]",[],"[Description, Princess Diana of an all-female ..."
106,No Time to Die,2021,"[89% liked this film, Google users]",[],"[Description, James Bond is enjoying a tranqui..."
107,The Hunger Games: Mockingjay - Part 1,2014,"[86% liked this film, Google users]",[],"[Description, After putting a permanent end to..."
115,Up,2009,"[89% liked this film, Google users]",[],"[Description, Carl, an old widower, goes off o..."


In [7]:
%%time

lst_scores2 = []
lst_like2 = []
lst_g_rev2 = []
lst_dist2 = []
lst_combined2 = []

loop_rand = random.randrange(10, 40)
loop_reset = 1
loop_counter = 1

driver = webdriver.Chrome('./chromedriver')
driver.get("https://www.google.com/")

# df_movies = missing_movies

for index, movie in missing_movies.iterrows():

    # Print stage in loop every 25 movies
    if loop_counter % 25 == 0:
        print('Loops completed: ', loop_counter)
    loop_counter += 1

    search = driver.find_element_by_name('q')
    search.send_keys(movie['Title']+ ' film '+  movie['Year'])
    search.send_keys(Keys.RETURN)

    # Movie review scores
    try:
        lst = google_scrape('//*[@id="kp-wp-tab-overview"]/div[1]/div/div/div[2]/div/div[1]/div[1]', driver)
        lst1_dict = [{'Title': movie['Title']
                    , 'Year': movie['Year']
                    , lst[1] : lst[0]
                    , lst[3] : lst[2]
                    , lst[5] : lst[4]}]
        lst_scores2.append(lst1_dict)
    except:
        pass

    # Did Google users like the movie?
    try:
        lst1_like = google_scrape('//*[@id="kp-wp-tab-overview"]/div[1]/div/div/div[2]/div/div[2]/div/div[1]/div', driver)
        lst_like2.append(lst1_like)
    except:
        pass

    # Google User Reviews Score
    for i in [2, 3, 4,5,6]:
        try: 
            path = '//*[@id="kp-wp-tab-overview"]/div[' + str(i) + ']/div/div/div/div/div/div/div/div[2]/div[2]'
            lst1_g_rev = google_scrape(path, driver)
            lst_g_rev2.append(lst1_g_rev)
            break
        except:
            pass    

    # Film description (from Google)
    try:
        lst1_dist = google_scrape('//*[@id="kp-wp-tab-overview"]/div[1]/div/div/div[2]/div/div[3]/div/div/div', driver)
        lst_dist2.append(lst1_dist)
    except:
        pass    

    # Combine everything except the agggregated movie reviews
    lst_combined2.append([movie['Title']
                        , movie['Year']
                        , lst1_like
                        , lst1_g_rev
                        , lst1_dist])
                        #, lst1_dict])

    # Empty lists
    lst1_like = []
    lst1_g_rev = []
    lst1_dist = []
    lst1_dict = []

    # Clear search bar
    driver.find_element_by_name('q').clear()

    # Wait random amount of time
    time.sleep(np.random.random() * 10)

    # Close and re-open driver at random interval to try to avoid Google's block
    loop_reset += 1
    if loop_reset >= loop_rand:
        loop_rand = random.randrange(20, 50)
        loop_reset = 1

        driver.close()
        driver = webdriver.Chrome('./chromedriver')
        driver.get("https://www.google.com/")

driver.close()


<timed exec>:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
<timed exec>:23: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
<timed exec>:78: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead


Loops completed:  25


<timed exec>:90: DeprecationWarning: executable_path has been deprecated, please pass in a Service object


Loops completed:  50
Wall time: 8min 16s


In [8]:
df_google2 = pd.DataFrame(lst_combined2, columns = col_names)

# Backup Movie list
df_google2.to_csv('bk_google_2_raw.csv', index = False)

df_google2.head()

,Title,Year,Google_Users,Google user ratings,movie description
0,Ice Age: Dawn of the Dinosaurs,2009,"[89% liked this film, Google users]","[4.6, 319 ratings]","[Description, Manny and Ellie are expecting th..."
1,Wonder Woman,2017,"[84% liked this film, Google users]","[4.1, 3411 ratings]","[Description, Princess Diana of an all-female ..."
2,No Time to Die,2021,"[89% liked this film, Google users]","[3.5, 17261 ratings]","[Description, James Bond is enjoying a tranqui..."
3,The Hunger Games: Mockingjay - Part 1,2014,"[86% liked this film, Google users]","[4.3, 489 ratings]","[Description, After putting a permanent end to..."
4,Up,2009,"[89% liked this film, Google users]","[4.8, 2506 ratings]","[Description, Carl, an old widower, goes off o..."


### Clean DataFrames
#### First run

In [9]:
# Clean up movies scores
df_scores_clean = movie_scores(lst_scores)
df_scores_clean.head()

,Title,Year,imdb_score,rotten_tom_score,metacritic_score
0,Avatar,2009,0.78,0.81,0.83
1,Avengers: Endgame,2019,0.84,0.94,NaN
2,Titanic,1997,0.78,0.89,0.75
3,Star Wars: Episode VII - The Force Awakens,2015,0.78,0.93,NaN
4,Avengers: Infinity War,2018,0.84,0.85,NaN


In [14]:
# Clean up data
df_google_clean = movies_clean(df_google)
df_google_clean.head()

,Title,Year,google_use_like,google_use_score,google_use_n_rev,movie_description
0,Avatar,2009,0.86,0.90,4274.0,"Jake, who is paraplegic, replaces his twin on ..."
1,Avengers: Endgame,2019,0.94,0.94,22212.0,"After Thanos, an intergalactic warlord, disint..."
2,Titanic,1997,0.94,0.94,8466.0,Seventeen-year-old Rose hails from an aristocr...
3,Star Wars: Episode VII - The Force Awakens,2015,0.80,0.66,2252.0,A new order threatens to destroy the New Repub...
4,Avengers: Infinity War,2018,0.92,0.94,5519.0,"The Avengers must stop Thanos, an intergalacti..."


In [15]:
# Merge to main dataframe
df_combined = df_google_clean.merge(df_scores_clean
                             , how = 'left'
                             , on = ['Title', 'Year'])
df_combined.head()

,Title,Year,google_use_like,google_use_score,google_use_n_rev,movie_description,imdb_score,rotten_tom_score,metacritic_score
0,Avatar,2009,0.86,0.90,4274.0,"Jake, who is paraplegic, replaces his twin on ...",0.78,0.81,0.83
1,Avengers: Endgame,2019,0.94,0.94,22212.0,"After Thanos, an intergalactic warlord, disint...",0.84,0.94,NaN
2,Titanic,1997,0.94,0.94,8466.0,Seventeen-year-old Rose hails from an aristocr...,0.78,0.89,0.75
3,Star Wars: Episode VII - The Force Awakens,2015,0.80,0.66,2252.0,A new order threatens to destroy the New Repub...,0.78,0.93,NaN
4,Avengers: Infinity War,2018,0.92,0.94,5519.0,"The Avengers must stop Thanos, an intergalacti...",0.84,0.85,NaN


#### Second Pass

In [17]:
# Movie Scores
df_scores_clean2 = movie_scores(lst_scores2)
df_scores_clean2.head()

,Title,Year,imdb_score,rotten_tom_score,metacritic_score
0,Ice Age: Dawn of the Dinosaurs,2009,0.69,0.46,NaN
1,Wonder Woman,2017,0.74,0.93,0.76
2,No Time to Die,2021,0.74,0.84,NaN
3,The Hunger Games: Mockingjay - Part 1,2014,0.66,0.69,NaN
4,Up,2009,0.82,0.98,NaN


In [18]:
# Main dataframe
df_google_clean2 = movies_clean(df_google2)
df_google_clean2.head()

,Title,Year,google_use_like,google_use_score,google_use_n_rev,movie_description
0,Ice Age: Dawn of the Dinosaurs,2009,0.89,0.92,319.0,Manny and Ellie are expecting their first chil...
1,Wonder Woman,2017,0.84,0.82,3411.0,Princess Diana of an all-female Amazonian race...
2,No Time to Die,2021,0.89,0.70,17261.0,James Bond is enjoying a tranquil life in Jama...
3,The Hunger Games: Mockingjay - Part 1,2014,0.86,0.86,489.0,"After putting a permanent end to the games, Ka..."
4,Up,2009,0.89,0.96,2506.0,"Carl, an old widower, goes off on an adventure..."


In [19]:
# Merge to main dataframe
df_combined2 = df_google_clean2.merge(df_scores_clean2
                                 , how = 'left'
                                 , on = ['Title', 'Year'])

df_combined2.head()

,Title,Year,google_use_like,google_use_score,google_use_n_rev,movie_description,imdb_score,rotten_tom_score,metacritic_score
0,Ice Age: Dawn of the Dinosaurs,2009,0.89,0.92,319.0,Manny and Ellie are expecting their first chil...,0.69,0.46,NaN
1,Wonder Woman,2017,0.84,0.82,3411.0,Princess Diana of an all-female Amazonian race...,0.74,0.93,0.76
2,No Time to Die,2021,0.89,0.70,17261.0,James Bond is enjoying a tranquil life in Jama...,0.74,0.84,NaN
3,The Hunger Games: Mockingjay - Part 1,2014,0.86,0.86,489.0,"After putting a permanent end to the games, Ka...",0.66,0.69,NaN
4,Up,2009,0.89,0.96,2506.0,"Carl, an old widower, goes off on an adventure...",0.82,0.98,NaN


### Merge All Data

In [20]:
# Create ID field for filtering
df_combined['id'] = df_combined['Title'] +'-'+ df_combined['Year']
df_combined2['id'] = df_combined2['Title'] +'-'+ df_combined2['Year']

# Append Original Google Book API search to secondary search
df_combined_final = pd.concat([df_combined.loc[~df_combined['id']
                                              .isin(df_combined2['id'] ), :]
                              ,df_combined2]
                              , axis = 0)

# No duplicates (1000 rows)
print(len(df_combined_final))

#df_combined_final.drop(columns = 'id', inplace = True)
df_combined_final.reset_index(inplace = True, drop = True)

# Output as CSV
df_combined_final.to_csv('bk_google_mojo.csv', index = False)

1000


In [21]:
df_combined_final.head()

,Title,Year,google_use_like,google_use_score,google_use_n_rev,movie_description,imdb_score,rotten_tom_score,metacritic_score,id
0,Avatar,2009,0.86,0.90,4274.0,"Jake, who is paraplegic, replaces his twin on ...",0.78,0.81,0.83,Avatar-2009
1,Avengers: Endgame,2019,0.94,0.94,22212.0,"After Thanos, an intergalactic warlord, disint...",0.84,0.94,NaN,Avengers: Endgame-2019
2,Titanic,1997,0.94,0.94,8466.0,Seventeen-year-old Rose hails from an aristocr...,0.78,0.89,0.75,Titanic-1997
3,Star Wars: Episode VII - The Force Awakens,2015,0.80,0.66,2252.0,A new order threatens to destroy the New Repub...,0.78,0.93,NaN,Star Wars: Episode VII - The Force Awakens-2015
4,Avengers: Infinity War,2018,0.92,0.94,5519.0,"The Avengers must stop Thanos, an intergalacti...",0.84,0.85,NaN,Avengers: Infinity War-2018
